## Imports

In [ ]:
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd().parent
sys.path.append(str(PROJECT_DIR))

# basics
import pandas as pd
import numpy as np
import json
import joblib

# models
from pycaret.regression import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import (
    train_test_split
)
from sklearn.metrics import (
    make_scorer,
    mean_squared_error
)

# utils
import os
from ppm.nodes.metrics_calculator import (
    metrics_calculate,
    show_results
)
from ppm.nodes.process_outputs import (
    ranking_output
)
from ppm.nodes.cross_validation import (
    cross_validation
)
from ppm.nodes.tunning_model import (
    perform_grid_search
)
from ppm.nodes.plots import (
    scatterplot_yx
)

## Parameters

In [ ]:
path_root = os.path.join("..","data")

path_primary = os.path.join(
    path_root, "03_primary"
)
path_model = os.path.join(
    path_root, "04_model"
)
path_encoders = os.path.join(
    path_model, "encoders"
)
path_model_final = os.path.join(
    path_model, "model"
)

file_path_metrics_features_selected = os.path.join(
    path_primary, "features_selected.json"
)
file_path_best_params = os.path.join(
    path_model, "best_params.json"
)
file_path_best_model = os.path.join(
    path_model, "model_tunned.joblib"
)
file_path_not_outliers = os.path.join(
    path_primary, "data_not_outliers.csv"
)
file_path_encoder_obj = os.path.join(
    path_encoders, "{}_encoder.joblib"
)
file_path_encoder_order = os.path.join(
    path_encoders, "encoders_orders.csv"
)
file_path_pycared_model = os.path.join(
    path_model_final, "{}.joblib"
)

In [ ]:
target = [
    "price"
]
cols_id_drop = [
    "cd_setor",
    "ID"
]
cols_drop = cols_id_drop + target

## Read dataset

In [ ]:
data_input = pd.read_csv(
    file_path_not_outliers, 
    index_col = 0
)

In [ ]:
if os.path.exists(file_path_metrics_features_selected):
    with open(file_path_metrics_features_selected, 'r') as json_file:
        features_selected = json.load(json_file)
    features_selected = features_selected["selected_features"]

In [ ]:
if os.path.exists(file_path_best_params):
    with open(file_path_best_params, 'r') as json_file:
        best_params = json.load(json_file)
    best_params = best_params["params"]
else:
    best_params = {
        "random_state": 42
    }

## Split data

In [ ]:
X = data_input[features_selected].copy()
y = data_input[target].copy()

## Tunning

In [ ]:
parameters = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 2, 3],
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2, 3],
    'max_features': ['auto'],
    "random_state": [42]
}

rf = RandomForestRegressor()

scorer = make_scorer(
    mean_squared_error,
    greater_is_better = False
)
best_model, best_params = perform_grid_search(
    X.values,
    y[target[0]].values,
    [rf],
    parameters,
    scorer,
    cv = 5
)

In [ ]:
best_model_params = {
    "params": best_params
}

In [ ]:
with open(file_path_best_params, 'w') as json_file:
    json.dump(
        best_model_params, 
        json_file
    )

In [ ]:
joblib.dump(
    best_model, 
    file_path_best_model
)

## Select best model (pycaret)

In [ ]:
X = data_input[target + features_selected].copy()

In [ ]:
regression_setup = setup(
    data = X,
    target = target[0], 
    verbose = False, 
    fold = 5
)

best_model = compare_models()
tuned_model = tune_model(best_model)
final_model = finalize_model(tuned_model)
predictions = predict_model(final_model)

evaluate_model(final_model)
df_metrics = pull()

In [ ]:
plot_model(final_model)

## Pipeline (encoders)

In [ ]:
if not os.path.exists(path_encoders):
    os.makedirs(path_encoders)

In [ ]:
encoders_order = []
for encoder_name, encoder_obj in final_model.steps[:(-1)]:
    joblib.dump(
        encoder_obj,
        file_path_encoder_obj.format(encoder_name)
    )
    encoders_order.append([encoder_name, file_path_encoder_obj.format(encoder_name)])
encoders_order = pd.DataFrame(encoders_order, columns = ["name", "url"])
encoders_order.to_csv(file_path_encoder_order, index = False)

## Pipeline (pycaret model tunned)

In [ ]:
for model_name, model_obj in [final_model.steps[(-1)]]:
    joblib.dump(
        model_obj,
        file_path_pycared_model.format(model_name)
    )